In [1]:
from IPython.display import display
import IPython
import pandas as pd
from pandas import Series
import numpy as np
from random import randrange
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import math
from typing import Any, List, Dict, AnyStr, Optional
from pathlib import Path
from glob import glob
from utils.dataset import MarketData, DataPreprocessor, Plotting
from utils.windowgenerator import WindowGenerator, compile_and_fit
import tensorflow as tf
import talib
%load_ext autoreload
%autoreload 2

In [19]:
eua_options_cols = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 
                    'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='EUA option-G363')
pachis_delta = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='25Delta')
option_ts = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='Option Time series')


In [36]:
display(eua_options.head()[eua_options_cols])

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
0,2024-10-21 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2024-10-17 00:00:00,195133.0,303171.0,785577.0,0.388123,1.553663
2,2024-10-16 00:00:00,190738.0,302671.0,782058.0,0.386846,1.586842
3,2024-10-15 00:00:00,191140.0,301316.0,780756.0,0.386782,1.576415
4,2024-10-14 00:00:00,190890.0,299416.0,778325.0,0.386484,1.568526


In [32]:
dec_cols = ["Date", "Hist Vol", "50D-Hist Vol", "50D", "25D Spread", "butterfly"]
pachis_delta_dec = pachis_delta[dec_cols]
pachis_delta_dec.columns = ["Date", "Hist Vol - 1Y", 
                        "iVol/Hist Vol Spread - Dec", 
                        "50 Delta iVol - Dec", 
                        "25Δ Risk Reversal (Call - Put) - Dec", 
                        "Butterfly - Dec"]

prompt_cols = ["Date.1", "50D.1", "25D Spread.1", "butterfly.1"]
pachis_delta_prompt = pachis_delta[prompt_cols]
pachis_delta_prompt.columns = ["Date", "50 Delta iVol - Prompt",
                               "25Δ Risk Reversal (Call - Put) - Prompt",
                               "Butterfly - Prompt"]

In [41]:
eua_columns = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options[eua_columns].isna().sum()

Date                                   0
Aggregate Put Open Interest  (R1)     15
Aggregate Call Open Interest  (R1)    15
Aggregate Open Interest  (L1)          2
OPTION OI%                            84
PUT/CALL OI                           92
dtype: int64

In [35]:
pachis_delta_dec.dropna()

,Date,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,Butterfly - Dec
0,2024-10-18 00:00:00,27.456,9.603,37.059,-3.036,1.2070
1,2024-10-17 00:00:00,27.527,9.114,36.641,-2.728,0.9720
2,2024-10-16 00:00:00,27.757,8.853,36.61,-3.004,1.0630
3,2024-10-15 00:00:00,27.384,9.556,36.94,-2.753,0.9485
4,2024-10-14 00:00:00,28.553,7.99,36.543,-2.276,0.9180
...,...,...,...,...,...,...
460,2023-06-01 00:00:00,40.724,11.274,51.998,-5.996,1.9830
461,2023-05-01 00:00:00,40.594,11.898,52.492,-5.973,1.8765
462,2023-04-01 00:00:00,40.956,12.611,53.567,-6.408,2.2720
463,2023-03-01 00:00:00,37.807,15.681,53.488,-5.371,2.0585
